Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [64]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [65]:
import funciones as f

In [66]:
#anio = int(input('Ingrese el año: '))
#mes = int(input('Ingrese el mes: '))

anio = 2024
mes = 1

In [67]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [68]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [69]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [70]:
#boletas

___________________________________________________________________________________________

In [71]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [72]:
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']].copy()
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = pd.to_numeric(bol['cantidad_anterior'], errors='coerce')
bol['cantidad_actual'] = pd.to_numeric(bol['cantidad_actual'], errors='coerce')
bol['precio_anterior'] = pd.to_numeric(bol['precio_anterior'], errors='coerce')
bol['precio_actual'] = pd.to_numeric(bol['precio_actual'], errors='coerce')
bol.loc[bol['cantidad_anterior'] == 0 | bol['cantidad_anterior'].isnull(), 'cantidad_anterior'] = bol['cantidad_actual']
bol.loc[bol['precio_anterior'] == 0 | bol['precio_anterior'].isnull(), 'precio_anterior'] = bol['precio_actual']

#bol['cantidad_actual'] = bol['cantidad_anterior']
#bol['precio_actual'] = bol['precio_anterior']

bol

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.0,1.0,145.0,145.0,NaN
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.0,1.0,250.0,250.0,NaN
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.0,1.0,325.0,400.0,NaN
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.0,1.0,1700.0,1700.0,NaN
4,1,0722401,7224013,Aceite de motor sintético,946.0,946.0,70.0,70.0,NaN
...,...,...,...,...,...,...,...,...,...
51522,8,0540101,5401011,Vajilla de porcelana/cerámica/vidrio para 4 pe...,1.0,1.0,115.0,115.0,NaN
51523,8,0540304,5403041,Vasos de plástico y melamina,1.0,1.0,1.0,1.0,NaN
51524,8,0561102,5611022,Bola (barra) de jabón para ropa (unidad),350.0,350.0,7.5,7.5,NaN
51525,8,0561906,5619061,Vela o candela blanca (unidad o paquete),1.0,1.0,7.0,7.0,NaN


In [73]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = f.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod['ponderacion_region'] = pd.to_numeric(pon_prod['ponderacion_region'], errors='coerce')
pon_prod['indice_anterior'] = pd.to_numeric(pon_prod['indice_anterior'], errors='coerce')
#pon_prod

In [74]:
#pon_prod['ponderacion_region'].dtype
#pon_prod['ponderacion_region'].isnull().sum()

Calculo del IPC

In [75]:
# Se calcula la variacion de las variedades
variacion = f.variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]
#variacion['var_prod'].dtype
#variacion

In [76]:
#variacion.to_excel('variacionpy.xlsx')

In [77]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())
relativo = pd.merge(relativo, pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
         on=['region', 'cod_prod'], how='left')
relativo
#relativo['ponderacion_region'].dtype

#relativo[relativo['ponderacion_region'] == 0]

,region,cod_prod,relativo,grupo_nombre,ponderacion_region,indice_anterior
0,1,0111101,1.001973,Arroz,0.399291,100.0
1,1,0111102,0.984308,Maíz,0.130256,100.0
2,1,0111201,0.980704,Harina de trigo,0.072427,100.0
3,1,0111202,1.002611,Harina de maíz,0.039212,100.0
4,1,0111203,1.025552,Harina para atoles,0.066483,100.0
...,...,...,...,...,...,...
2648,8,1321101,1.000000,Anillos,0.068566,100.0
2649,8,1321104,1.000000,Relojes de pulsera,0.065485,100.0
2650,8,1329102,1.000000,Ataúdes,0.065264,100.0
2651,8,1390901,1.000000,Servicios legales y financieros,0.194022,100.0


In [78]:
#nulos = relativo['ponderacion_region'].isnull()
#relativo[nulos]
#relativo.to_excel('relativopy.xlsx')

In [79]:
#promedio ponderado
prom_pon = relativo
prom_pon['var_ac'] = prom_pon['relativo']*prom_pon['ponderacion_region']
prom_pon = prom_pon[prom_pon['var_ac'] != 0]
## lo unico que se me ocurre es hacer otros dataframes agrupados, uno para la suma de la var ac, otro para la suma de las ponderaciones y asi y luego unirlos

#prom_pon['sum_var_ac'] = prom_pon['var_ac'].sum()
prom_pon01 = prom_pon.groupby(['region', 'cod_prod', 'grupo_nombre']).agg(
    sum_var_ac=('var_ac', 'sum'),
    sum_ponderacion=('ponderacion_region', 'sum')
    ).reset_index()
prom_pon = prom_pon.merge(prom_pon01, on=['region', 'cod_prod', 'grupo_nombre'], how='left')
prom_pon['prom_pon'] = prom_pon['sum_var_ac']/prom_pon['sum_ponderacion']

#prom_pon
#prom_pon01
#prom_pon[prom_pon['var_ac'] == 0]
#prom_pon01[prom_pon01['sum_var_ac'] == 0]

In [80]:
#prom_pon.to_excel('promponpy.xlsx')

Índice de precios por región

In [81]:
#índice productos
prom_pon['ind_prod'] = prom_pon['indice_anterior']*prom_pon['prom_pon']
ind_prod = f.cod_grupos(prom_pon).dropna()

ind_prod01 = ind_prod[['region', 'cod_prod', 'ind_prod']].copy()
#dist100['ind_prod'].dtype

In [82]:
#indice subclase
f.indices(ind_prod,'cod_subclase')

,region,grupo_codigo,ind
0,1,01111,99.762807
1,1,01112,100.226588
2,1,01113,98.459317
3,1,01114,99.949574
4,1,01115,99.123358
...,...,...,...
1141,8,13131,100.000000
1142,8,13132,100.000000
1143,8,13211,100.000000
1144,8,13291,100.000000


In [83]:
#indice clase
f.indices(ind_prod,'cod_clase')

,region,grupo_codigo,ind
0,1,0111,98.716955
1,1,0112,101.108204
2,1,0113,100.408477
3,1,0114,101.169079
4,1,0115,100.263995
...,...,...,...
633,8,1312,99.994771
634,8,1313,100.000000
635,8,1321,100.000000
636,8,1329,100.000000


In [84]:
#indice grupo
f.indices(ind_prod,'cod_grupo')

,region,grupo_codigo,ind
0,1,011,99.980107
1,1,012,99.477330
2,1,013,100.000000
3,1,021,99.972425
4,1,023,97.737569
...,...,...,...
308,8,105,100.000000
309,8,111,100.180147
310,8,131,99.995696
311,8,132,100.000000


In [85]:
#indice division
f.indices(ind_prod,'cod_div')

,region,grupo_codigo,ind
0,1,01,99.938864
1,1,02,99.358785
2,1,03,100.122457
3,1,04,99.817034
4,1,05,100.131999
...,...,...,...
92,8,08,100.000000
93,8,09,100.144987
94,8,10,100.073198
95,8,11,100.180147


Índice a nivel republica

In [86]:
#indice productos
f.indicerep(ind_prod,'cod_prod')

,cod_prod,ind_prod_rep
0,0111101,100.055327
1,0111102,100.003704
2,0111201,99.022178
3,0111202,100.368525
4,0111203,99.947755
...,...,...
415,1390901,100.000000
416,1390902,100.107068
417,1390903,100.000000
418,1390904,100.000000


In [87]:
#indice subclase
f.indicerep(ind_prod,'cod_subclase')

,cod_subclase,ind_prod_rep
0,01111,100.024325
1,01112,100.028281
2,01113,99.265884
3,01114,99.562244
4,01115,100.079714
...,...,...
159,13131,100.026347
160,13132,100.000000
161,13211,100.039135
162,13291,100.000000


In [88]:
#indice clase
f.indicerep(ind_prod,'cod_clase')

,cod_clase,ind_prod_rep
0,0111,99.461303
1,0112,100.947559
2,0113,100.551809
3,0114,99.800455
4,0115,100.208403
...,...,...
90,1312,99.855142
91,1313,100.023370
92,1321,100.039135
93,1329,100.000000


In [89]:
#indice grupo
f.indicerep(ind_prod,'cod_grupo')

,cod_grupo,ind_prod_rep
0,011,100.087531
1,012,99.916405
2,013,100.096266
3,021,99.941997
4,023,99.146150
5,031,99.974697
6,032,99.911567
7,041,100.042963
8,043,99.929160
9,044,100.437236


In [90]:
#indice division
f.indicerep(ind_prod,'cod_div')

,cod_div,ind_prod_rep
0,01,100.072747
1,02,99.780381
2,03,99.959601
3,04,99.392958
4,05,99.916585
5,06,100.045342
6,07,100.815192
7,08,100.075528
8,09,99.987512
9,10,100.150497


Indice general

In [91]:
#indice producos a nivel nacional
def indicenac(df):
    indice = df.groupby('region').apply(lambda x: pd.Series({
        'sum1': (x['ponderacion_region'] * x['ind_prod']).sum(),
        'sum2': x['ponderacion_region'].sum()
    })).reset_index()

    indice['ind_prod_rep'] = indice['sum1'] / indice['sum2']
    indice = indice[['region', 'ind_prod_rep']]

    return indice



In [92]:
indicenac(ind_prod)

,region,ind_prod_rep
0,1,100.043231
1,2,100.252046
2,3,100.012513
3,4,99.713581
4,5,100.054915
5,6,100.101849
6,7,99.840457
7,8,100.268743


In [93]:
def calculate_replicated_indicator(df):
    sum1 = (df['ponderacion_region'] * df['ind_prod']).sum()
    sum2 = df['ponderacion_region'].sum()

    result = pd.DataFrame({
        'sum1': [sum1],
        'sum2': [sum2]
    })

    result['ind_rep'] = result['sum1'] / result['sum2']

    return result

In [94]:
calculate_replicated_indicator(ind_prod)

,sum1,sum2,ind_rep
0,69750.296087,697.24129,100.037529
